##Updated HLCM estimation

Sam Maurer and Paul Waddell, August 2015

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os; os.chdir('..');os.chdir('..')
import models
import orca
# to try variations, create a new yaml file and run this to register it
# to save variations, create a new yaml file and run this to register it
from urbansim.utils import misc
from urbansim import accounts
from urbansim.developer import sqftproforma
from urbansim_defaults import models
from urbansim_defaults import utils

In [3]:
%%capture
orca.run(["neighborhood_vars"])

In [4]:
%%capture
orca.run(["rsh_simulate", "rrh_simulate", "nrh_simulate"])

## HLCM for owners

In [ ]:
# The model expression is in hlcm_owner.yaml and it's registered in models.py

In [23]:
@orca.step("hlcm_owner_estimate_NEW6")
def hlcm_owner_estimate_NEW(households, residential_units, unit_aggregations):
    return utils.lcm_estimate("hlcm_owner_NEW6.yaml", households, "unit_id",
                              residential_units, unit_aggregations)

orca.run(["hlcm_owner_estimate_NEW6"])

Running step 'hlcm_owner_estimate_NEW6'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -68234.807
Log-liklihood at convergence: -67911.511
Log-liklihood Ratio: 0.005

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| np.log1p(residential_price)     |    0.339    |   0.087    |  3.872  |
| np.log1p(sqft_per_unit)         |    0.124    |   0.043    |  2.847  |
| np.log1p(ave_lot_size_per_unit) |    0.559    |   0.141    |  3.960  |
| ave_income                      |    -0.413   |   0.043    |  -9.514 |
| jobs                            |    -0.029   |   0.006    |  -4.725 |
| autoPeakTotal                   |    -0.687   |   0.065    | -10.599 |
| transitPeakTotal                |    0.000    |   0.005    |  0.037  |
| autoOffPeakRetail               |    0.684    |   0.065    |  10.450 |
+-----------------------------

## ols

In [24]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [25]:
# a is buildings
a = orca.get_injectable('unit_aggregations')[0].to_frame()[['node_id','residential_price','sqft_per_unit','zone_id']]
a.head()

,node_id,residential_price,sqft_per_unit,zone_id
building_id,,,,
1,11584,392.395039,794,893
2,11584,284.880588,1600,893
3,11585,307.013146,1334,893
4,11585,319.807287,1220,893
5,11585,331.039145,892,893


In [26]:
# b is logsums
b = orca.get_injectable('unit_aggregations')[2].to_frame()[['autoPeakTotal', 'transitPeakTotal' , 'autoOffPeakRetail']]
b['b_id'] = b.index
b.head()

,autoPeakTotal,transitPeakTotal,autoOffPeakRetail,b_id
taz,,,,
1,13.0616,11.1182,10.5312,1
2,13.0557,10.8372,10.5185,2
3,13.0308,10.7516,10.4998,3
4,13.0801,11.1391,10.5458,4
5,13.0613,11.0483,10.5325,5


In [27]:
import pandas as pd
a=pd.merge(a, b, left_on='zone_id', right_on='b_id')
a.head()

,node_id,residential_price,sqft_per_unit,zone_id,autoPeakTotal,transitPeakTotal,autoOffPeakRetail,b_id
0,11584,392.395039,794,893,12.0346,6.8494,9.9296,893
1,11584,284.880588,1600,893,12.0346,6.8494,9.9296,893
2,11585,307.013146,1334,893,12.0346,6.8494,9.9296,893
3,11585,319.807287,1220,893,12.0346,6.8494,9.9296,893
4,11585,331.039145,892,893,12.0346,6.8494,9.9296,893


In [28]:
c = orca.get_table('nodes').to_frame()[['ave_lot_size_per_unit','ave_income','ave_hhsize','jobs']]
c['c_id'] = c.index
c.head()

,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,c_id
8,9.760910,10.962228,1.388186,7.051473,8
9,9.667663,10.947813,1.384454,7.129230,9
10,9.939272,10.901420,1.399366,7.164987,10
11,9.697163,10.905808,1.481605,7.112618,11
12,9.671385,10.884773,1.486836,7.108247,12


In [29]:
import pandas as pd
a=pd.merge(a, c, left_on='node_id', right_on='c_id')
a.head()

,node_id,residential_price,sqft_per_unit,zone_id,autoPeakTotal,transitPeakTotal,autoOffPeakRetail,b_id,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,c_id
0,11584,392.395039,794,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
1,11584,284.880588,1600,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
2,11584,379.145588,856,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
3,11584,344.759122,1054,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
4,11584,379.145588,856,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584


# OLS to get residual as a pandas series

In [30]:
a.head()

,node_id,residential_price,sqft_per_unit,zone_id,autoPeakTotal,transitPeakTotal,autoOffPeakRetail,b_id,ave_lot_size_per_unit,ave_income,ave_hhsize,jobs,c_id
0,11584,392.395039,794,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
1,11584,284.880588,1600,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
2,11584,379.145588,856,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
3,11584,344.759122,1054,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584
4,11584,379.145588,856,893,12.0346,6.8494,9.9296,893,8.197712,10.517467,1.510924,6.793979,11584


In [31]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [32]:
model = ols(" np.log1p(residential_price)~ np.log1p(sqft_per_unit)+ autoPeakTotal + jobs + transitPeakTotal + autoOffPeakRetail + np.log1p(ave_lot_size_per_unit)+ ave_income", data=a).fit()

In [33]:
print model.summary()

                                 OLS Regression Results                                
Dep. Variable:     np.log1p(residential_price)   R-squared:                       0.105
Model:                                     OLS   Adj. R-squared:                  0.105
Method:                          Least Squares   F-statistic:                 3.079e+04
Date:                         Tue, 01 Mar 2016   Prob (F-statistic):               0.00
Time:                                 14:25:35   Log-Likelihood:            -3.4040e+06
No. Observations:                      1841288   AIC:                         6.808e+06
Df Residuals:                          1841280   BIC:                         6.808e+06
Df Model:                                    7                                         
Covariance Type:                     nonrobust                                         
                                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------

In [34]:
c = model.resid
print type(c)
print len(c)
print len(a)

<class 'pandas.core.series.Series'>
1841288
1841288


In [35]:
orca.add_column('buildings', 'residual', c)

In [36]:
orca.get_table('buildings').to_frame().describe()

,parcel_id,residential_units,residential_sqft,non_residential_sqft,building_sqft,stories,year_built,redfin_sale_year,building_type_id,residential_price,...,sqft_per_job,residential_rent,residual,lot_size_per_unit,vacant_job_spaces,vacant_residential_units,node_id,base_price_per_sqft,job_spaces,sqft_per_unit
count,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288,1841288.000000,1841288.000000,...,1794938.000000,1841288.000000,1.841287e+06,1.841288e+06,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288.000000,1841288.000000
mean,1012181.855456,1.512957,2555.752946,1189.012710,3555.140902,1.214768,1965.502413,2012,1.742778,373.780476,...,405.491496,2.171768,-3.219367e-07,5.021285e+04,0.369858,0.071124,125537.823235,290.026722,2.027393,1825.815012
std,592975.033646,6.302781,29116.786991,23914.761611,34053.945683,0.557437,25.014873,0,2.425943,154.304960,...,49.563977,1.057609,1.536861e+00,8.047316e+05,25.709189,0.684230,88114.671062,150.074783,35.532501,1052.321849
min,26.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1790.000000,2012,0.000000,0.000000,...,355.000000,0.000000,-1.227344e+01,0.000000e+00,-25530.000000,0.000000,8.000000,27.442371,0.000000,400.000000
25%,496121.750000,1.000000,1200.000000,0.000000,1260.000000,1.000000,1952.000000,2012,1.000000,308.641746,...,400.000000,1.765167,2.137861e-01,3.640196e+03,0.000000,0.000000,56018.000000,204.374383,0.000000,1200.000000
50%,1002024.500000,1.000000,1674.000000,0.000000,1790.000000,1.000000,1966.000000,2012,1.000000,379.832878,...,400.000000,2.108319,4.068881e-01,6.113516e+03,0.000000,0.000000,121294.000000,266.049383,0.000000,1582.000000
75%,1531020.250000,1.000000,2400.000000,0.000000,2496.000000,1.000000,1983.000000,2012,1.000000,458.638329,...,400.000000,2.579237,5.745391e-01,9.032667e+03,0.000000,0.000000,168436.000000,344.938164,0.000000,2115.000000
max,2048562.000000,1909.000000,14580000.000000,20405104.000000,14580000.000000,59.000000,2015.000000,2012,16.000000,1472.557766,...,1161.000000,21.624895,8.184041e+00,4.469411e+08,5841.000000,294.000000,354685.000000,9413.020833,11985.000000,6000.000000


In [37]:
orca.get_table('buildings').to_frame().columns

Index([u'parcel_id', u'residential_units', u'residential_sqft',
       u'non_residential_sqft', u'building_sqft', u'stories', u'year_built',
       u'redfin_sale_year', u'building_type_id', u'residential_price',
       u'non_residential_price', u'zone_id', u'general_type', u'sqft_per_job',
       u'residential_rent', u'residual', u'lot_size_per_unit',
       u'vacant_job_spaces', u'vacant_residential_units', u'node_id',
       u'base_price_per_sqft', u'job_spaces', u'sqft_per_unit'],
      dtype='object')

In [67]:
# import os
# import uuid
# import warnings

# import numpy as np
# import orca
# import pandas as pd
# from urbansim.utils import misc
# import yaml
# from urbansim_defaults import utils
# #import utils


# @orca.injectable('settings', cache=True)
# def settings():
#     with open(os.path.join(misc.configs_dir(), "settings.yaml")) as f:
#         settings = yaml.load(f)
#         # monkey patch on the settings object since it's pretty global
#         # but will also be available as injectable
#         orca.settings = settings
#         return settings
# @orca.injectable("unit_aggregations")
# def aggregations(settings):
#     return [orca.get_table(tbl) for tbl in settings["unit_aggregations"]]

In [38]:
orca.get_injectable('unit_aggregations')[0].to_frame().columns

Index([u'parcel_id', u'residential_units', u'residential_sqft',
       u'non_residential_sqft', u'building_sqft', u'stories', u'year_built',
       u'redfin_sale_year', u'building_type_id', u'residential_price',
       u'non_residential_price', u'zone_id', u'general_type', u'sqft_per_job',
       u'residential_rent', u'residual', u'lot_size_per_unit',
       u'vacant_job_spaces', u'vacant_residential_units', u'node_id',
       u'base_price_per_sqft', u'job_spaces', u'sqft_per_unit'],
      dtype='object')

In [39]:
@orca.step("hlcm_owner_estimate_NEW5")
def hlcm_owner_estimate_NEW(households, residential_units, unit_aggregations):
    return utils.lcm_estimate("hlcm_owner_NEW5.yaml", households, "unit_id",
                              residential_units, unit_aggregations)

orca.run(["hlcm_owner_estimate_NEW5"])

Running step 'hlcm_owner_estimate_NEW5'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -68096.652
Log-liklihood at convergence: -67773.334
Log-liklihood Ratio: 0.005

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| np.log1p(residential_price)     |    0.480    |   0.088    |  5.439  |
| np.log1p(sqft_per_unit)         |    0.199    |   0.044    |  4.537  |
| np.log1p(ave_lot_size_per_unit) |    0.733    |   0.143    |  5.116  |
| residual                        |    -0.004   |   0.005    |  -0.658 |
| ave_income                      |    -0.458   |   0.044    | -10.372 |
| jobs                            |    -0.026   |   0.006    |  -4.391 |
| autoPeakTotal                   |    -0.751   |   0.066    | -11.407 |
| transitPeakTotal                |    -0.002   |   0.005    |  -0.295 |
| autoOffPeakRetail           

In [40]:
orca.run(["hlcm_owner_estimate"])

Running step 'hlcm_owner_estimate'
LCM RESULTS FOR SEGMENT 1

Null Log-liklihood: -68184.150
Log-liklihood at convergence: -65771.889
Log-liklihood Ratio: 0.035

+---------------------------------+-------------+------------+---------+
| Component                       | Coefficient | Std. Error | T-Score |
+---------------------------------+-------------+------------+---------+
| np.log1p(residential_price)     |    2.214    |   0.091    |  24.349 |
| np.log1p(sqft_per_unit)         |    1.017    |   0.045    |  22.609 |
| np.log1p(ave_lot_size_per_unit) |    -0.171   |   0.147    |  -1.163 |
| ave_income                      |    -1.479   |   0.048    | -30.552 |
| np.log1p(persons * ave_hhsize)  |    3.000    |   0.022    | 135.926 |
| np.log1p(white * pct_white)     |    3.000    |   0.009    | 342.400 |
| np.log1p(black * pct_black)     |    3.000    |   0.011    | 267.691 |
| np.log1p(hisp * pct_hisp)       |    3.000    |   0.007    | 421.546 |
| np.log1p(asian * pct_asian)     |